In [ ]:
import xarray as xr
import warnings
warnings.simplefilter('ignore') # filter some warning messages
from glob import glob

# Read ATOMIC Saildrone data via openDaP  --- Not working ---

In [ ]:
url = 'https://podaac-opendap.jpl.nasa.gov/opendap/allData/insitu/L2/saildrone/Atomic//saildrone-gen_5-atomic_eurec4a_2020-sd1061-20200117T000000-20200302T235959-1_minutes-v1.1595707641693.nc'

ds_usv = xr.open_dataset(url)

# try using podaac drive (z:)  --- This works ----

In [ ]:
url = 'Z:/allData/insitu/L2/saildrone/Atomic/saildrone-gen_5-atomic_eurec4a_2020-sd1061-20200117T000000-20200302T235959-1_minutes-v1.1595707641693.nc'

ds_usv = xr.open_dataset(url)

ds_usv

# try using file downloaded locally (from opendap link) -- this works --

In [ ]:
url = './../data/saildrone-gen_5-atomic_eurec4a_2020-sd1061-20200117T000000-20200302T235959-1_minutes-v1.1595707641693.nc'

ds_usv = xr.open_dataset(url)

ds_usv

# clean up usv file for interpolation
- calculate date range for files to read in

In [ ]:
ds_usv = ds_usv.rename({'latitude':'lat','longitude':'lon'}).isel(trajectory=0).swap_dims({'obs':'time'})

ds_usv

# Read in SMAP from mapped podaac drive (z:)

In [ ]:
#RSS
#get list of all filenames in directory
syr = str(ds_usv.time[0].dt.year.data).zfill(4)
adir = 'Z:/SalinityDensity/smap/L3/RSS/V4/8day_running/SCI/'+syr+'/*/*.nc'
files = glob(adir)
print('number of file:',len(files))

# read in data
ds = xr.open_mfdataset(files,combine='nested',concat_dim='time')
ds.coords['lon'] = (ds.coords['lon'] + 180) % 360 - 180
ds = ds.sortby(ds.lon)
ds.close()  
ds_rss = ds    


In [ ]:
#JPL
#get list of all filenames in directory
syr = str(ds_usv.time[0].dt.year.data).zfill(4)
adir = 'Z:/SalinityDensity/smap/L3/JPL/V4.3/8day_running/'+syr+'/*/*.nc'
files = glob(adir)
print('number of file:',len(files))
  
# read in data
ds = xr.open_mfdataset(files,combine='nested',concat_dim='time')
ds.coords['lon'] = (ds.coords['lon'] + 180) % 360 - 180
ds = ds.sortby(ds.lon)
ds.close()  
ds_jpl = ds

# Collocate SMAP RSS 8day

In [ ]:
%%time
ds_usv['lat'] = ds_usv.lat.interpolate_na(dim='time',method='linear').ffill(dim='time').bfill(dim='time')
ds_usv['lon'] = ds_usv.lon.interpolate_na(dim='time',method='linear').ffill(dim='time').bfill(dim='time')
tem = ds_usv.lat.ffill(dim='time')
tem = ds_usv.lat.bfill(dim='time')
t1,t2=ds_usv.time.min().data-np.timedelta64(8,'D'),ds_usv.time.max().data+np.timedelta64(8,'D')
x1,x2=ds_usv.lon.min().data,ds_usv.lon.max().data
y1,y2=ds_usv.lat.min().data,ds_usv.lat.max().data
print(t1,t2)
ds_sat = ds.sel(time=slice(t1,t2),lat=slice(y1,y2),lon=slice(x1,x2)).load()   
ds_interp = ds_sat.interp(time=ds_usv.time,lat=ds_usv.lat,lon=ds_usv.lon,method='linear')#.interp(method='nearest')
#fout = dir_out+name+'_RSS8dy'+'.nc'
#ds_interp.to_netcdf(fout)

# Collocation SMAP JPL 8day

In [ ]:
#JPL
adir = 'F:/data/sat_data/smap/SSS/L3/JPL/V4.3/8day_running/**/**/*4.3.nc'
files = [x for x in glob(adir)]
print('number of file:',len(files))

ds = xr.open_mfdataset(files,combine='nested',concat_dim='time')
ds = ds.rename({'latitude':'lat','longitude':'lon'})
ds = ds.sortby(ds.lat)
ds.close()  
ds

In [ ]:
%%time
ds_usv['lat'] = ds_usv.lat.interpolate_na(dim='time',method='linear').ffill(dim='time').bfill(dim='time')
ds_usv['lon'] = ds_usv.lon.interpolate_na(dim='time',method='linear').ffill(dim='time').bfill(dim='time')
tem = ds_usv.lat.ffill(dim='time')
tem = ds_usv.lat.bfill(dim='time')
t1,t2=ds_usv.time.min().data-np.timedelta64(8,'D'),ds_usv.time.max().data+np.timedelta64(8,'D')
x1,x2=ds_usv.lon.min().data,ds_usv.lon.max().data
y1,y2=ds_usv.lat.min().data,ds_usv.lat.max().data
print(t1,t2)
ds_sat = ds.sel(time=slice(t1,t2),lat=slice(y1,y2),lon=slice(x1,x2)).load()   
ds_interp = ds_sat.interp(time=ds_usv.time,lat=ds_usv.lat,lon=ds_usv.lon,method='linear')#.interp(method='nearest')
#fout = dir_out+name+'_JPL8dy'+'.nc'
#ds_interp.to_netcdf(fout)